In [53]:
import sys
sys.path.append("..")

from data_transformation.db_env import DbEnv, db

# Native libraries
import os
import math
# Essential Libraries
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import numpy as np
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Algorithms
from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

https://www.kaggle.com/izzettunc/introduction-to-time-series-clustering

# 1.Preprocessing

In [54]:
#get song_num list
# 지수 추가해서 그래프에서 비교 가능하게 해주기, 클러스터링 기준?

conn, cursor = DbEnv().connect_sql()
sql = "SELECT DISTINCT num FROM daily_music_cow"
num_list = DbEnv().get_data_from_table(cursor, sql)
num_list = [item[0] for item in num_list]
print(num_list)

[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 167, 168, 169, 171, 173, 174, 175, 176, 177, 178, 179, 180, 183, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 199, 202, 204, 206, 207, 208, 211, 212, 216, 218, 219, 221, 224, 225, 230, 231, 233, 237, 238, 240, 258, 261, 262, 265, 267, 274, 311, 322, 345, 373, 382, 420, 460, 461, 462, 464, 465, 466, 467, 468, 469, 470, 471, 472, 474, 475, 477, 478, 479, 480, 481, 482, 483, 484, 485

In [55]:
df_price, df_mcpi = pd.DataFrame(), pd.DataFrame()

# find shortest date length to fit all songs
for num in num_list:
    sql = "SELECT date, price FROM daily_music_cow WHERE num = %s" % num
    df_temp = db(cursor, sql).dataframe
    df_temp = df_temp.set_index('date')
    df_temp.columns = ["%d" % num]

    df_price = pd.concat([df_price, df_temp], axis=1)

print(df_price)
df_price.to_pickle("../storage/df_price.pkl")

               26     27     28     29     30     31     32     33     34  \
date                                                                        
2021-06-03  29000  26000  24000  52800  13500  16100  10300  14500  36400   
2021-06-04  29000  26000  24000  52800  13500  16100  10300  14500  36400   
2021-06-05  29000  26000  23700  52800  13500  16100  10800  14500  36400   
2021-06-06  28500  26000  23500  52800  13500  16100  10800  14500  36400   
2021-06-07  29100  26000  24900  52800  13500  15100  10800  14500  36400   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
2021-11-23  29000  43000  29800  90700  19100  70100  15100  37200  40000   
2021-11-24  28500  43000  29900  90700  19100  70000  15000  37200  40000   
2021-11-25  27900  43000  29800  90700  19100  70000  15000  37200  40000   
2021-11-26  30000  43000  31200  90700  19100  70000  15000  37200  40000   
2021-11-27  27000  43000  29100  90700  19000  70000  16100  37200  40000   

In [56]:
sql = "SELECT date, price FROM daily_mcpi"
df_mcpi = db(cursor, sql).dataframe
df_mcpi = df_mcpi.set_index('date')
df_mcpi.columns = [0]

print(df_mcpi)
df_mcpi.to_pickle("../storage/df_mcpi.pkl")

                 0
date              
2021-12-08  246.43
2021-12-07  247.01
2021-12-06  246.48
2021-12-05  246.18
2021-12-04  245.84
...            ...
2019-03-24   83.54
2019-03-23   83.59
2019-03-22   83.44
2019-03-21   82.88
2019-03-20   82.59

[1990 rows x 1 columns]


In [59]:
# 결측치 제거를 통해 date 맞춰주기
df_price = pd.read_pickle('../storage/df_price.pkl')
df_mcpi = pd.read_pickle('../storage/df_mcpi.pkl')

df_price = pd.concat([df_mcpi, df_price], axis=1)

df_price_droped = df_price.dropna(axis=0)
df_price_droped = df_price_droped.sort_index(ascending=True)
list_price_droped = list(df_price_droped.columns)

print(df_price_droped)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
# MinMaxScaler로 정규화
array_price_scaled = MinMaxScaler().fit_transform(df_price_droped)
df_price_scaled = pd.DataFrame(array_price_scaled)
df_price_scaled.columns = list_price_droped
df_price_scaled.index = df_price_droped.index
print(df_price_scaled)

# num=0(MCPI) 지수 df에서 제거
df_mcpi = df_price_scaled[df_price_scaled.columns[0]]
df_price_scaled = df_price_scaled.drop(df_price_scaled.columns[0], axis='columns')
array_price_scaled = np.delete(array_price_scaled, 0, 1)

print(df_mcpi)

# 2. Clustering

In [ ]:
array_price_scaled = np.transpose(array_price_scaled)

print(df_mcpi)

## (1) SOM raw data

In [ ]:
som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(array_price_scaled))))


som = MiniSom(som_x, som_y,len(array_price_scaled[0]), sigma=0.3, learning_rate=0.1)

som.random_weights_init(array_price_scaled)
som.train(array_price_scaled, 50000)

In [ ]:
def plot_som_series_averaged_center(som_x, som_y, win_map):
    fig, axs = plt.subplots(som_x,som_y,figsize=(25,25))
    fig.suptitle('Clusters')
    for x in range(som_x):
        for y in range(som_y):
            cluster = (x,y)
            if cluster in win_map.keys():
                for series in win_map[cluster]:
                    axs[cluster].plot(series,c="gray",alpha=0.5) 
                axs[cluster].plot(df_mcpi, c="red")
                axs[cluster].plot(np.average(np.vstack(win_map[cluster]),axis=0),c="blue")
            cluster_number = x*som_y+y+1
            axs[cluster].set_title(f"Cluster {cluster_number}")

    plt.show()

In [ ]:
win_map = som.win_map(array_price_scaled)

plot_som_series_averaged_center(som_x, som_y, win_map)

In [ ]:
cluster_c = []
cluster_n = []
for x in range(som_x):
    for y in range(som_y):
        cluster = (x,y)
        if cluster in win_map.keys():
            cluster_c.append(len(win_map[cluster]))
        else:
            cluster_c.append(0)
        cluster_number = x*som_y+y+1
        cluster_n.append(f"Cluster {cluster_number}")

plt.figure(figsize=(25,5))
plt.title("Cluster Distribution for SOM")
plt.bar(cluster_n,cluster_c)
plt.show()

<br>시계열이 시각적으로도 잘 구분을 못함<br/>
<br>위아래로의 등락이 너무 Chaotic하기 때문이라고 판단.<br/>
<br>특히 뮤직카우의 경우 유동성이 충분하지 않기 때문에 적절한 균형가격에 Optimize되지 않는 문제<br/>
<br>Smoothing을 통해 다시 분류<br/>

 # (2) SOM holtwinters weight 0.3

In [ ]:
#smoothing 0.3
array_price = df_price_droped.to_numpy()
array_price = np.transpose(array_price)

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt

array_price_smooth = np.empty((0, 145))

for x in array_price:
    model_smooth = SimpleExpSmoothing(x)
    fit_smooth = model_smooth.fit(smoothing_level=.3)
    array_price_smooth = np.append(array_price_smooth, fit_smooth.fittedvalues.reshape(1, 145), axis=0)

print(array_price_smooth)

In [ ]:
# MinMaxScaler로 정규화
array_price_smooth = np.transpose(array_price_smooth)
array_price_smooth_scaled = MinMaxScaler().fit_transform(array_price_smooth)
df_price_smooth_scaled = pd.DataFrame(array_price_smooth_scaled)
df_price_smooth_scaled.columns = list_price_droped
df_price_smooth_scaled.index = df_price_droped.index
print(df_price_smooth_scaled)

# num=0(MCPI) 지수 df에서 제거
df_mcpi = df_price_smooth_scaled[df_price_smooth_scaled.columns[0]]
df_price_scaled = df_price_smooth_scaled.drop(df_price_smooth_scaled.columns[0], axis='columns')
array_price_smooth_scaled = np.delete(array_price_smooth_scaled, 0, 1)

print(df_mcpi)

In [ ]:
array_price_smooth_scaled = np.transpose(array_price_smooth_scaled)

som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(array_price_smooth_scaled))))


som = MiniSom(som_x, som_y,len(array_price_smooth_scaled[0]), sigma=0.3, learning_rate=0.1)

som.random_weights_init(array_price_smooth_scaled)
som.train(array_price_smooth_scaled, 50000)

In [ ]:
win_map = som.win_map(array_price_smooth_scaled)

plot_som_series_averaged_center(som_x, som_y, win_map)

## (3) SOM holtwinters weight 0.5

In [ ]:
#smoothing 0.5
array_price = df_price_droped.to_numpy()
array_price = np.transpose(array_price)

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt

array_price_smooth = np.empty((0, 145))

for x in array_price:
    model_smooth = SimpleExpSmoothing(x)
    fit_smooth = model_smooth.fit(smoothing_level=.5)
    array_price_smooth = np.append(array_price_smooth, fit_smooth.fittedvalues.reshape(1, 145), axis=0)
print(array_price_smooth)

In [ ]:
# MinMaxScaler로 정규화
array_price_smooth = np.transpose(array_price_smooth)
array_price_smooth_scaled = MinMaxScaler().fit_transform(array_price_smooth)
df_price_smooth_scaled = pd.DataFrame(array_price_smooth_scaled)
df_price_smooth_scaled.columns = list_price_droped
df_price_smooth_scaled.index = df_price_droped.index
print(df_price_smooth_scaled)

# num=0(MCPI) 지수 df에서 제거
df_mcpi = df_price_smooth_scaled[df_price_smooth_scaled.columns[0]]
df_price_scaled = df_price_smooth_scaled.drop(df_price_smooth_scaled.columns[0], axis='columns')
array_price_smooth_scaled = np.delete(array_price_smooth_scaled, 0, 1)


print(df_mcpi)

In [ ]:
array_price_smooth_scaled = np.transpose(array_price_smooth_scaled)

som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(array_price_smooth_scaled))))


som = MiniSom(som_x, som_y,len(array_price_smooth_scaled[0]), sigma=0.3, learning_rate=0.1)

som.random_weights_init(array_price_smooth_scaled)
som.train(array_price_smooth_scaled, 50000)

In [ ]:
win_map = som.win_map(array_price_smooth_scaled)

plot_som_series_averaged_center(som_x, som_y, win_map)

In [ ]:
cluster_map = []
for idx in range(len(array_price_smooth_scaled)):
    winner_node = som.winner(array_price_smooth_scaled[idx])
    cluster_map.append((list_price_droped[idx],f"Cluster {winner_node[0]*som_y+winner_node[1]+1}"))

df_cluster = pd.DataFrame(cluster_map,columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

In [ ]:
cluster_c = []
cluster_n = []
for x in range(som_x):
    for y in range(som_y):
        cluster = (x,y)
        if cluster in win_map.keys():
            cluster_c.append(len(win_map[cluster]))
        else:
            cluster_c.append(0)
        cluster_number = x*som_y+y+1
        cluster_n.append(f"Cluster {cluster_number}")

plt.figure(figsize=(25,5))
plt.title("Cluster Distribution for SOM")
plt.bar(cluster_n,cluster_c)
plt.show()

시계열에서 MCPI 효과 제거할 것

In [ ]:
conn = DbEnv().connect_mongo('music_cow', 'daily_music_cow')

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 1"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 2"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 3"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 4"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 5"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 6"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 7"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 8"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 9"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 10"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 11"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 12"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 13"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 14"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 15"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

In [ ]:
list_cluster = df_cluster[df_cluster.Cluster == "Cluster 16"].index.tolist()
print(list_cluster)
for cl in list_cluster:
    list_song = conn.find({'num': int(cl)})
    for x in list_song:
        print('title:', x['song_title'], '---', 'artist:', x['song_artist'])

지수효과 제거해줄 것

## 2. Dynamic Time Warping Barycenter Averaging (DBA)

In [32]:
from copy import deepcopy
from scipy import interpolate
import numpy as np
from dtaidistance import dtw
import matplotlib.pyplot as plt
from _plotly_future_ import v4_subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [33]:
NUM_OF_TRAJECTORIES = 200
MIN_LEN_OF_TRAJECTORY = 16
MAX_LEN_OF_TRAJECTORY = 40
THRESHOLD = 0.50

https://towardsdatascience.com/time-series-hierarchical-clustering-using-dynamic-time-warping-in-python-c8c9edf2fda5

In [ ]:
array_price_scaled = deepcopy(array_price_scaled)
distanceMatrixDictionary = {}
iteration = 1
while True:
   distanceMatrix = np.empty((len(array_price_scaled), len(array_price_scaled),))
   distanceMatrix[:] = np.nan
   
   for index1, (filter1, trajectory1) in enumerate(trajectories.items()):
      tempArray = []
      
      for index2, (filter2, trajectory2) in enumerate(trajectories.items()):
         
         if index1 > index2:
            continue
         
         elif index1 == index2:
            continue
         
         else:
            unionFilter = filter1 + filter2
            sorted(unionFilter)
            
            if unionFilter in distanceMatrixDictionary.keys():
               distanceMatrix[index1][index2] = distanceMatrixDictionary.get(unionFilter)
               
               continue
            
            metric = []
            for subItem1 in trajectory1:
               
               for subItem2 in trajectory2:
                  metric.append(dtw.distance(subItem1, subItem2, psi=1))
            
            metric = max(metric)
            
            distanceMatrix[index1][index2] = metric
            distanceMatrixDictionary[unionFilter] = metric